In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score

In [55]:
df = pd.read_csv("/kaggle/input/dataset04/CreditCard4.csv")

# Feature selection

In [56]:
# Categorical columns
cat_col = [col for col in df.columns if df[col].dtype == 'object']
print('Categorical columns :',cat_col)
# Numerical columns
num_col = [col for col in df.columns if df[col].dtype != 'object']
print('Numerical columns :',num_col)

Categorical columns : ['Gender', 'MaritalStatus', 'CardColour', 'CardType', 'Domain', 'Customer_City_Address']
Numerical columns : ['AcountNumber', 'CVV', 'CustomerAge', 'Amount', 'AverageIncomeExpendicture', 'Outcome']


In [57]:
from scipy.stats import chi2_contingency

def drop_non_significant_cols(df, target_column, significance_level=0.05):
    dropped_columns = []
    
    for column in df.select_dtypes(include='object'):
        contingency_table = pd.crosstab(df[column], df[target_column])
        chi2, p, _, _ = chi2_contingency(contingency_table)
        
        if p > significance_level:
            df.drop(column, axis=1, inplace=True)
            dropped_columns.append(column)
    
    return df, dropped_columns


df, dropped_columns = drop_non_significant_cols(df, 'Outcome')

print("Dropped Columns:", dropped_columns)

Dropped Columns: ['Gender', 'MaritalStatus', 'CardColour', 'Customer_City_Address']


In [58]:
df.head()

,AcountNumber,CVV,CustomerAge,CardType,Domain,Amount,AverageIncomeExpendicture,Outcome
0,1275734409,364,28.0,Verve,Local,129282,170919,0
1,1271246193,401,25.0,Verve,International,574384,329353,1
2,1242290165,266,21.0,Visa,International,190766,292922,0
3,1245478185,402,26.0,Visa,Local,130395,145444,0
4,1258212072,334,28.0,Verve,International,685145,295990,1


# fill null values

In [59]:
df.fillna(method='pad',inplace=True)

In [60]:
df.isnull().sum()

AcountNumber                 0
CVV                          0
CustomerAge                  0
CardType                     0
Domain                       0
Amount                       0
AverageIncomeExpendicture    0
Outcome                      0
dtype: int64

# one hot encoding

In [61]:
import warnings
warnings.filterwarnings('ignore')

In [62]:
print(df.CardType.unique())
print(df.Domain.unique())

['Verve' 'Visa' 'MasterCard']
['Local' 'International']


In [63]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

In [64]:
encoder_df = pd.DataFrame(encoder.fit_transform(df[['CardType']]).toarray())
df = df.join(encoder_df)
df.rename(columns={0: 'Type_MasterCard', 1: 'Type_Verve',2: 'Type_Visa'}, inplace=True)
df.drop('CardType', axis=1, inplace=True)

In [65]:
df.head()

,AcountNumber,CVV,CustomerAge,Domain,Amount,AverageIncomeExpendicture,Outcome,Type_MasterCard,Type_Verve,Type_Visa
0,1275734409,364,28.0,Local,129282,170919,0,0.0,1.0,0.0
1,1271246193,401,25.0,International,574384,329353,1,0.0,1.0,0.0
2,1242290165,266,21.0,International,190766,292922,0,0.0,0.0,1.0
3,1245478185,402,26.0,Local,130395,145444,0,0.0,0.0,1.0
4,1258212072,334,28.0,International,685145,295990,1,0.0,1.0,0.0


In [66]:
encoder_df = pd.DataFrame(encoder.fit_transform(df[['Domain']]).toarray())
df = df.join(encoder_df)
df.rename(columns={0: 'Domain_International', 1: 'Domain_Local'}, inplace=True)
df.drop('Domain', axis=1, inplace=True)

In [67]:
df.head()

,AcountNumber,CVV,CustomerAge,Amount,AverageIncomeExpendicture,Outcome,Type_MasterCard,Type_Verve,Type_Visa,Domain_International,Domain_Local
0,1275734409,364,28.0,129282,170919,0,0.0,1.0,0.0,0.0,1.0
1,1271246193,401,25.0,574384,329353,1,0.0,1.0,0.0,1.0,0.0
2,1242290165,266,21.0,190766,292922,0,0.0,0.0,1.0,1.0,0.0
3,1245478185,402,26.0,130395,145444,0,0.0,0.0,1.0,0.0,1.0
4,1258212072,334,28.0,685145,295990,1,0.0,1.0,0.0,1.0,0.0


In [68]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [69]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('Outcome', axis=1), df['Outcome'],
    test_size=0.20, random_state=42)

# Random under

In [70]:
from imblearn.under_sampling import RandomUnderSampler

# Display class distribution before undersampling
print("Class distribution in y_train before undersampling:\n", y_train.value_counts())

# Apply Random UnderSampling to the training data
undersampler = RandomUnderSampler(random_state=42)
X_train_undersampled, y_train_undersampled = undersampler.fit_resample(X_train, y_train)

# Display class distribution after undersampling
print("Class distribution in y_train after undersampling:\n", y_train_undersampled.value_counts())

Class distribution in y_train before undersampling:
 Outcome
1    21971
0     7706
Name: count, dtype: int64
Class distribution in y_train after undersampling:
 Outcome
0    7706
1    7706
Name: count, dtype: int64


In [71]:
model =  DecisionTreeClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

DecisionTreeClassifier()

In [72]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8552560646900269
Precision : 0.9385520178462786
Recall    : 0.8571957769957399
F1-score  : 0.8960309777347532


In [73]:
model =  RandomForestClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

RandomForestClassifier()

In [74]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8636118598382749
Precision : 0.996604029884537
Recall    : 0.8153361733654381
F1-score  : 0.8969030154849227


In [75]:
model = AdaBoostClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

AdaBoostClassifier()

In [76]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8563342318059299
Precision : 0.9762585183556826
Recall    : 0.8225597332839415
F1-score  : 0.8928427824688381


In [77]:
model = GradientBoostingClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

GradientBoostingClassifier()

In [78]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8633423180592992
Precision : 0.9995443153337891
Recall    : 0.8125578810890906
F1-score  : 0.8964037597057621


# Near miss 1

In [79]:
from imblearn.under_sampling import NearMiss

# Display class distribution before undersampling
print("Class distribution in y_train before undersampling:\n", y_train.value_counts())

# Apply NearMiss-1 to the training data
nearmiss = NearMiss(version=1)
X_train_undersampled, y_train_undersampled = nearmiss.fit_resample(X_train, y_train)

# Display class distribution after undersampling
print("Class distribution in y_train after NearMiss-1 undersampling:\n", y_train_undersampled.value_counts())

Class distribution in y_train before undersampling:
 Outcome
1    21971
0     7706
Name: count, dtype: int64
Class distribution in y_train after NearMiss-1 undersampling:
 Outcome
0    7706
1    7706
Name: count, dtype: int64


In [80]:
model =  DecisionTreeClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

DecisionTreeClassifier()

In [81]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8409703504043127
Precision : 0.8888479262672812
Recall    : 0.8931283571031673
F1-score  : 0.8909830007390983


In [82]:
model =  RandomForestClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

RandomForestClassifier()

In [83]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8584905660377359
Precision : 0.919706620343563
Recall    : 0.8825708464530468
F1-score  : 0.9007561436672967


In [84]:
model = AdaBoostClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

AdaBoostClassifier()

In [85]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8477088948787062
Precision : 0.8940372900129223
Recall    : 0.8970179662900537
F1-score  : 0.8955251479289941


In [86]:
model = GradientBoostingClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

GradientBoostingClassifier()

In [87]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8650943396226415
Precision : 0.9556568586821385
Recall    : 0.8542322652343026
F1-score  : 0.9021026894865526


# Near miss 2

In [88]:
from imblearn.under_sampling import NearMiss

# Display class distribution before undersampling
print("Class distribution in y_train before undersampling:\n", y_train.value_counts())

# Apply NearMiss-1 to the training data
nearmiss = NearMiss(version=2)
X_train_undersampled, y_train_undersampled = nearmiss.fit_resample(X_train, y_train)

# Display class distribution after undersampling
print("Class distribution in y_train after NearMiss-1 undersampling:\n", y_train_undersampled.value_counts())

Class distribution in y_train before undersampling:
 Outcome
1    21971
0     7706
Name: count, dtype: int64
Class distribution in y_train after NearMiss-1 undersampling:
 Outcome
0    7706
1    7706
Name: count, dtype: int64


In [89]:
model =  DecisionTreeClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

DecisionTreeClassifier()

In [90]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.5181940700808625
Precision : 0.9122965641952984
Recall    : 0.3737729209112799
F1-score  : 0.5302851136512942


In [91]:
model =  RandomForestClassifier()
model.fit(X_train_undersampled, y_train_undersampled)


RandomForestClassifier()

In [92]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.6597035040431267
Precision : 0.9605769230769231
Recall    : 0.5551027968142248
F1-score  : 0.703603709355558


In [93]:
model = AdaBoostClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

AdaBoostClassifier()

In [94]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.46846361185983826
Precision : 0.882291119285339
Recall    : 0.31098351546582703
F1-score  : 0.4598740071213367


In [95]:
model = GradientBoostingClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

GradientBoostingClassifier()

In [96]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.48274932614555255
Precision : 0.951417004048583
Recall    : 0.30468605297277274
F1-score  : 0.4615600448933783


# Near miss 3

In [97]:
from imblearn.under_sampling import NearMiss

# Display class distribution before undersampling
print("Class distribution in y_train before undersampling:\n", y_train.value_counts())

# Apply NearMiss-1 to the training data
nearmiss = NearMiss(version=3)
X_train_undersampled, y_train_undersampled = nearmiss.fit_resample(X_train, y_train)

# Display class distribution after undersampling
print("Class distribution in y_train after NearMiss-1 undersampling:\n", y_train_undersampled.value_counts())

Class distribution in y_train before undersampling:
 Outcome
1    21971
0     7706
Name: count, dtype: int64
Class distribution in y_train after NearMiss-1 undersampling:
 Outcome
0    7706
1    5755
Name: count, dtype: int64


In [98]:
model =  DecisionTreeClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

DecisionTreeClassifier()

In [99]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8455525606469003
Precision : 0.8928505449842971
Recall    : 0.8951657714391554
F1-score  : 0.8940066592674806


In [100]:
model =  RandomForestClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

RandomForestClassifier()

In [101]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8588948787061995
Precision : 0.9288529759558534
Recall    : 0.8729394332283756
F1-score  : 0.9000286450873675


In [102]:
model = AdaBoostClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

AdaBoostClassifier()

In [103]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8475741239892183
Precision : 0.9158222915042868
Recall    : 0.8705315799222079
F1-score  : 0.8926027917576679


In [104]:
model = GradientBoostingClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

GradientBoostingClassifier()

In [105]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8629380053908355
Precision : 0.9843059239610964
Recall    : 0.8247823671050194
F1-score  : 0.8975108334173134


# ENN

In [106]:
from imblearn.under_sampling import EditedNearestNeighbours

# Display class distribution before undersampling
print("Class distribution in y_train before undersampling:\n", y_train.value_counts())

# Apply Edited Nearest Neighbors undersampling to the training data
enn = EditedNearestNeighbours(sampling_strategy='auto')
X_train_undersampled, y_train_undersampled = enn.fit_resample(X_train, y_train)

# Display class distribution after undersampling
print("Class distribution in y_train after Edited Nearest Neighbors undersampling:\n", y_train_undersampled.value_counts())

Class distribution in y_train before undersampling:
 Outcome
1    21971
0     7706
Name: count, dtype: int64
Class distribution in y_train after Edited Nearest Neighbors undersampling:
 Outcome
1    16661
0     7706
Name: count, dtype: int64


In [107]:
model =  DecisionTreeClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

DecisionTreeClassifier()

In [108]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8547169811320755
Precision : 0.969571832210389
Recall    : 0.8262641229857381
F1-score  : 0.8922


In [109]:
model =  RandomForestClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

RandomForestClassifier()

In [110]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8602425876010782
Precision : 0.9970373746581586
Recall    : 0.8103352472680126
F1-score  : 0.8940431184223971


In [111]:
model = AdaBoostClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

AdaBoostClassifier()

In [112]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8576819407008086
Precision : 0.9826628139586575
Recall    : 0.8188553435821448
F1-score  : 0.8933117801576076


In [113]:
model = GradientBoostingClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

GradientBoostingClassifier()

In [114]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.860512129380054
Precision : 0.9984010963910461
Recall    : 0.8095943693276533
F1-score  : 0.8941393065357471


# Tomek

In [115]:
from imblearn.under_sampling import TomekLinks

# Display class distribution before undersampling
print("Class distribution in y_train before undersampling:\n", y_train.value_counts())

# Apply Tomek Links undersampling to the training data
tomek_links_undersampler = TomekLinks(sampling_strategy='auto')
X_train_undersampled, y_train_undersampled = tomek_links_undersampler.fit_resample(X_train, y_train)

# Display class distribution after undersampling
print("Class distribution in y_train after Tomek Links undersampling:\n", y_train_undersampled.value_counts())

Class distribution in y_train before undersampling:
 Outcome
1    21971
0     7706
Name: count, dtype: int64
Class distribution in y_train after Tomek Links undersampling:
 Outcome
1    20384
0     7706
Name: count, dtype: int64


In [116]:
model =  DecisionTreeClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

DecisionTreeClassifier()

In [117]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8479784366576819
Precision : 0.9073049780659928
Recall    : 0.8810890905723282
F1-score  : 0.8940048862995676


In [118]:

model =  RandomForestClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

RandomForestClassifier()

In [119]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8615902964959569
Precision : 0.9645133871653209
Recall    : 0.8407112428227449
F1-score  : 0.8983671449777337


In [120]:
model = AdaBoostClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

AdaBoostClassifier()

In [121]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8524258760107817
Precision : 0.9300559552358113
Recall    : 0.8620114836080756
F1-score  : 0.8947419013746035


In [122]:

model = GradientBoostingClassifier()
model.fit(X_train_undersampled, y_train_undersampled)

GradientBoostingClassifier()

In [123]:
# Make predictions on test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy   :", accuracy)
precision = precision_score(y_test, y_pred)
print("Precision :", precision)
recall = recall_score(y_test, y_pred)
print("Recall    :", recall)
F1_score = f1_score(y_test, y_pred)
print("F1-score  :", F1_score)

Accuracy   : 0.8618598382749326
Precision : 0.9847074468085106
Recall    : 0.8229301722541211
F1-score  : 0.8965795580667945
